# A basic `TensorFlow` installation check

Pre-reqs: If nothing is working right, check this notebook, [../jims_ml_notebook_env.py].

A sanity check call to `tensorflow` to make sure it finds and loads the library. NOTE: this does not do any tensor processing:

In [3]:
import tensorflow as tf

print(f"A TF computation: {tf.add(1, 5).numpy()}")

hello = tf.constant('Hello, TensorFlow!')
print(f"A TensorFlow string constant: {hello.numpy()}")

A TF computation: 6
A TensorFlow string constant: b'Hello, TensorFlow!'


# Of special `import` for `keras`
Keras notions should be imported from `keras.*`, not from `tensorflow`.

When coding to the conventional API:

In [4]:

# Imports when using the conventional class-centric Keras API

# from keras.api.datasets import mnist
from keras.api.models import Sequential
from keras.api.layers import Input, Dense




When coding to the newer functional Keras API:


In [5]:
import keras  # has keras.Model keras.Input etc


Now we load a dataset from the convenient tensorflow_datasets python package.

In [6]:
import importlib
import jkcsoft.ml.tensorflow_utils

importlib.reload(jkcsoft.ml.tensorflow_utils)

from jkcsoft.ml.tensorflow_utils import dump_dataset_info, display_dataset_info


# Validates datasets package install and lists voluminous available datasets
dump_dataset_info()

# Create an array of 5 most popular datasets
most_popular_datasets = ['mnist', 'cifar10', 'imdb_reviews', 'fashion_mnist', 'coco']
print(f"Most popular datasets: {most_popular_datasets}")

for dataset_name in most_popular_datasets:
    display_dataset_info(dataset_name)


Total datasets available: 1304
A few available datasets: ['abstract_reasoning', 'accentdb', 'aeslc', 'aflw2k3d', 'ag_news_subset', 'ai2_arc', 'ai2_arc_with_ir', 'ai2dcaption', 'aloha_mobile', 'amazon_us_reviews'] ...
Most popular datasets: ['mnist', 'cifar10', 'imdb_reviews', 'fashion_mnist', 'coco']

Dataset: mnist
Description: The MNIST database of handwritten digits.
Features: FeaturesDict({
    'image': Image(shape=(28, 28, 1), dtype=uint8),
    'label': ClassLabel(shape=(), dtype=int64, num_classes=10),
})
Supervised keys: ('image', 'label')
Splits: {'test': <SplitInfo num_examples=10000, num_shards=1>, 'train': <SplitInfo num_examples=60000, num_shards=1>}
Dataset size: 21.00 MiB bytes
Citation: @article{lecun2010mnist,
  title={MNIST handwritten digit database},
  author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
  journal={ATT Labs [Online]. Available: http://yann.lecun.com/exdb/mnist},
  volume={2},
  year={2010}
}

Dataset: cifar10
Description: The CIFAR-10 dataset con

# Setup training and test datums to be used in the next two examples

The common datums are used to compare the use of the two Keras APIs.


In [7]:
from jkcsoft.ml.sandbox.tf_tinker import ModelTest, RunBatch

# define a batch include a common dataset for all tests
run_batch = RunBatch()
run_batch.x_train = tf.constant([[1.0], [2.0], [3.0], [4.0]])
run_batch.y_train = tf.constant([[2.0], [4.0], [6.0], [8.0]])
run_batch.x_test = tf.constant([[5.0], [6.0]])
run_batch.y_test = tf.constant([[10.0], [12.0]])

# A simple _conventional_ Keras pipeline with hard-coded TensorFlow datasets

In [8]:
# Define a model using the conventional Keras class API
model = Sequential([
    Input(shape=(1,)),  # Input layer for single-dimensional input
    Dense(32, activation="relu"),  # kera.src.core.dense.Dense
    Dense(16, activation="sigmoid"),
    Dense(16, activation="softmax"),
    Dense(1, activation="linear")
])

test_common_dense = ModelTest().set_description("Common Dense Sequence").set_model(model)
test_common_dense.set_fit_epochs(10)

# invoke model compile here
test_common_dense.compile_model(
    optimizer="adam",
    loss="mean_squared_error",
    metrics=[
        "mean_absolute_error",
        "mean_absolute_percentage_error",
        "mean_squared_error",
        "root_mean_squared_error",
        "accuracy",
        "precision",
        "recall",
        "auc",
        "binary_accuracy"
    ])

# test_common_dense.model.fit()

run_batch.add_test(test_common_dense)

# run all models against the same batch dataset
run_batch.run_all()

print('\nTest accuracy:', run_batch.results[0].eval_results)

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Testing Model: Common Dense Sequence
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │            64 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 881 (3.44 KB)

 Trainable params: 881 (3.44 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.0000e+00 - auc: 0.0000e+00 - binary_accuracy: 0.0000e+00 - loss: 30.5274 - mean_absolute_error: 5.0532 - mean_absolute_percentage_error: 101.4089 - mean_squared_error: 30.5274 - precision: 0.0000e+00 - recall: 0.0000e+00 - root_mean_squared_error: 5.5252
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.0000e+00 - auc: 0.0000e+00 - binary_accuracy: 0.0000e+00 - loss: 30.4612 - mean_absolute_error: 5.0468 - mean_absolute_percentage_error: 101.2473 - mean_squared_error: 30.4612 - precision: 0.0000e+00 - recall: 0.0000e+00 - root_mean_squared_error: 5.5192
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.0000e+00 - auc: 0.0000e+00 - binary_accuracy: 0.0000e+00 - loss: 30.3949 - mean_absolute_error: 5.0404 - mean_absolute_percentage_error: 101.0854 - mean_squared_error: 30.3949 - precision: 0.0000e+00 - recall: 0.0000e+00 - root_mean_squared_error: 5.5132
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step

# A simple pipeline with the Keras _functional_ API

In [9]:
import keras
from keras import layers

keras.backend.clear_session()

# Keep the input tensor
inputs_fn = keras.Input(shape=(1,))

print(f"inputs shape: {inputs_fn.shape}")

print(f"inputs dtype: {inputs_fn.dtype}")

# pipe input
d1 = layers.Dense(32, activation="relu")
x = d1(inputs_fn)

# pipe thru proc layers
d2 = layers.Dense(16, activation="sigmoid")
x = d2(x)
d3 = layers.Dense(8, activation="softmax")
x = d3(x)

# use final layer to reduce to output
final_layer = layers.Dense(1, activation="linear")
outputs_fn = final_layer(x)

functional_model = keras.Model(inputs=inputs_fn, outputs=outputs_fn, name="functional_model_1")

print(f"functional_model summary: {functional_model.summary()}")

functional_model.compile(
    optimizer="adam",
    loss="mean_squared_error",
    metrics=[
        "mean_absolute_error",
        "mean_absolute_percentage_error",
        "mean_squared_error",
        "root_mean_squared_error",
        "accuracy",
        "precision",
        "recall",
        "auc",
        "binary_accuracy"
    ]
)

functional_model.fit(run_batch.x_train, run_batch.y_train, epochs=3, verbose=1)

loss_and_metrics = functional_model.evaluate(run_batch.x_test, run_batch.y_test, verbose=2)

print(f"Test eval results: {loss_and_metrics}")

inputs shape: (None, 1)
inputs dtype: float32


Model: "functional_model_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │            64 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 737 (2.88 KB)

 Trainable params: 737 (2.88 KB)

 Non-trainable params: 0 (0.00 B)

functional_model summary: None
Epoch 1/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.0000e+00 - auc: 0.0000e+00 - binary_accuracy: 0.0000e+00 - loss: 32.6887 - mean_absolute_error: 5.2570 - mean_absolute_percentage_error: 106.5344 - mean_squared_error: 32.6887 - precision: 0.0000e+00 - recall: 0.0000e+00 - root_mean_squared_error: 5.7174
Epoch 2/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.0000e+00 - auc: 0.0000e+00 - binary_accuracy: 0.0000e+00 - loss: 32.6156 - mean_absolute_error: 5.2503 - mean_absolute_percentage_error: 106.3648 - mean_squared_error: 32.6156 - precision: 0.0000e+00 - recall: 0.0000e+00 - root_mean_squared_error: 5.7110
Epoch 3/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.0000e+00 - auc: 0.0000e+00 - binary_accuracy: 0.0000e+00 - loss: 32.5422 - mean_absolute_error: 5.2435 - mean_absolute_percentage_error: 106.1944 - mean_squared_error: 32.5422 - precision: 0.0000e+00 - recall: 0.0000e+00 - root_mean_squared_error: 5.7046
1/1 - 0s - 428ms/ste

## A more readable functional pipeline declaration

In [10]:
from jkcsoft.ml import keras_utils

importlib.reload(keras_utils)

proc_layers = [d1, d2, d3, final_layer]

composite_model = keras_utils.compose_model(inputs_fn, proc_layers, verbose=False)

print(f"composite_model summary: {composite_model.summary()}")

composite_model.compile(
    optimizer="adam",
    loss="mean_squared_error",
    metrics=[
#        "mean_absolute_error",
        "mean_absolute_percentage_error",
#        "mean_squared_error",
#        "root_mean_squared_error",
#        "accuracy",
#        "precision",
#        "recall",
#        "auc",
#        "binary_accuracy"
    ]
)

composite_model.fit(run_batch.x_train, run_batch.y_train, epochs=3, verbose=1)

loss_and_metrics = composite_model.evaluate(run_batch.x_test, run_batch.y_test, verbose=2)

print(f"Test eval results: {loss_and_metrics}")

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │            64 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 737 (2.88 KB)

 Trainable params: 737 (2.88 KB)

 Non-trainable params: 0 (0.00 B)

composite_model summary: None
Epoch 1/3


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 520ms/step - loss: 32.4684 - mean_absolute_percentage_error: 106.0233
Epoch 2/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 32.3943 - mean_absolute_percentage_error: 105.8515
Epoch 3/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 32.3198 - mean_absolute_percentage_error: 105.6789


1/1 - 0s - 88ms/step - loss: 127.3175 - mean_absolute_percentage_error: 102.1859
Test eval results: [127.31748962402344, 102.18589782714844]


## A _functional_ model loaded into our `RunBatch` apparatus

In [11]:
keras.backend.clear_session()

test_common_dense_fn = ModelTest().set_description("Common Dense Sequence").set_model(functional_model)
test_common_dense_fn.set_fit_epochs(10)
test_common_dense_fn.compile_model(
    optimizer="adam",
    loss="mean_squared_error",
    metrics=[
#        "mean_absolute_error",
        "mean_absolute_percentage_error",
#        "mean_squared_error",
#        "root_mean_squared_error",
#        "accuracy",
#        "precision",
#        "recall",
#        "auc",
#        "binary_accuracy"
    ]
)

test_common_dense_fn.model.summary()

test_common_dense_fn.model.fit(run_batch.x_train, run_batch.y_train, epochs=10, verbose=1)

run_batch.add_test(test_common_dense_fn)

run_batch.run_all()

for i, result in enumerate(run_batch.results):
    print(f"Test {i} eval results: {result.eval_results}")


Model: "functional_model_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │            64 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 737 (2.88 KB)

 Trainable params: 737 (2.88 KB)

 Non-trainable params: 0 (0.00 B)

...

 Total params: 2,213 (8.65 KB)

 Trainable params: 737 (2.88 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,476 (5.77 KB)

Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 31.4808 - mean_absolute_percentage_error: 103.7387
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 31.4029 - mean_absolute_percentage_error: 103.5587
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 31.3248 - mean_absolute_percentage_error: 103.3782
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 31.2464 - mean_absolute_percentage_error: 103.1969
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 31.1677 - mean_absolute_percentage_error: 103.0147
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 31.0886 - mean_absolute_percentage_error: 102.8316
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 31.0091 - mean_absolute_percentage_error: 102.6475
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 30.9292 - mean_absolute_percentage_error: 102.4623
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 30.8487 - mean_absolute_percentage_error: 102.2759
Epoch 10/10
1/1 ━━━━━━━━━━━━